In [1]:
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import re
import numpy as np

In [2]:
spcpath = "../Data/SPC_TripSummaries_44Districts.xlsx"
spcxls = pd.ExcelFile(spcpath)
sheet_list = spcxls.sheet_names  # see all sheet names
tot_list = sheet_list[0:3]
hwy_list = sheet_list[3:6]
trans_list = sheet_list[6:9]

#for i, s in enumerate(sheet_list):
#    print('{}: {}'.format(i,s))

In [6]:
trans_hbw_tb = pd.read_excel(spcpath, 'T7_HBW_TranPers')

In [3]:
print(tot_list)
print(hwy_list)
print(trans_list)

['T1_HBW_TotPers', 'T2_HBO_TotPers', 'T3_NHB_TotPers']
['T4_HBW_HwyVeh', 'T5_HBO_HwyVeh', 'T6_NHB_HwyVeh']
['T7_HBW_TranPers', 'T8_HBO_TranPers', 'T9_NHB_TranPers']


In [7]:
dis_names = list(trans_hbw_tb.iloc[5:49,1])
dis_dict = { i : dis_names[i-1] for i in range(1, len(dis_names)+1 ) }
dis_dict

{1: 'Pittsburgh Chateau Area',
 2: 'Pittsburgh Lower North Side',
 3: 'Pittsburgh Upper North Side',
 4: 'Allegheny County Northwest',
 5: 'Allegheny County North (remainder)',
 6: 'Hazelwood',
 7: 'Squirrel Hill',
 8: 'Oakland',
 9: 'South Side',
 10: 'Shadyside',
 11: 'Waterfront (West Homestead Homestead Munhall(part))',
 12: 'Downtown',
 13: 'East Liberty',
 14: 'Strip District',
 15: 'Pittsburgh East (remainder)',
 16: 'Hays Lincoln Place',
 17: 'Verona Oakmont Penn Hills',
 18: 'Plum Borough',
 19: 'Monroeville Pitcairn Turtle Creek',
 20: 'Wilkinsburg Wilkins Braddock and vicinity',
 21: 'Wilmerding Wall North Versailles',
 22: 'McKeesport White Oak',
 23: 'Lincoln Liberty Port Vue',
 24: 'Elizabeth Forward South Versailles',
 25: 'Munhall Whitaker Duquesne',
 26: 'Baldwin Brentwood Whitehall West Mifflin',
 27: 'Allegheny County Southeast (remainder)',
 28: 'Allegheny County West',
 29: 'Armstrong County East',
 30: 'Armstrong County West',
 31: 'Beaver County South',
 32: 'Bea

In [8]:
def extract_tab_2020(excel, sheet_name):
    sheet_df = pd.read_excel(excel, sheet_name)
    sub_df = sheet_df.iloc[5:49,4:48]
    sub_df.columns = dis_dict.values()
    return(sub_df)

In [10]:
def extract_tab_2045(excel, sheet_name):
    sheet_df = pd.read_excel(excel, sheet_name)
    sub_df = sheet_df.iloc[61:105,4:48]
    sub_df.columns = dis_dict.values()
    return(sub_df)

In [11]:
spc_df_dict_2020 = {sheet_name: extract_tab_2020(spcxls, sheet_name) for sheet_name in spcxls.sheet_names}

In [12]:
spc_df_dict_2045 = {sheet_name: extract_tab_2045(spcxls, sheet_name) for sheet_name in spcxls.sheet_names}

In [13]:
for sheet_name, sheet_data in spc_df_dict_2020.items():
    fname = '../Outputs/spc20_' + sheet_name +'.csv'
    #print(fname)
    #sheet_data.to_csv(fname, index = None, header=True)

In [14]:
for sheet_name, sheet_data in spc_df_dict_2045.items():
    fname = '../Outputs/spc45' + sheet_name +'.csv'
    #print(fname)
    #sheet_data.to_csv(fname, index = None, header=True)

In [15]:
def combo_tab(excel_dict, tab_list, district_dictionary, offset):
    #print(tab_list[0])
    trans_all = excel_dict[tab_list[0]]
    #print(trans_all.iloc[:2,:2])
    for tab in tab_list[1:]:
        #print(tab)
        #print(excel_dict[tab].iloc[:2,:2])
        trans_all = trans_all.add(excel_dict[tab], fill_value=0)
    colnames = list(trans_all.columns)
    trans_all.reset_index(inplace=True)
    trans_all['index'] = trans_all['index']-offset
    trans_all['dist_origin'] = trans_all['index'].map(district_dictionary)
    trans_all = trans_all[['index', 'dist_origin']+colnames]
    return(trans_all)

In [16]:
tot_2020_all = combo_tab(spc_df_dict_2020, tot_list, dis_dict, 4)
tot_2045_all = combo_tab(spc_df_dict_2045, tot_list, dis_dict, 60)

trans_2020_all = combo_tab(spc_df_dict_2020, trans_list, dis_dict, 4)
trans_2045_all = combo_tab(spc_df_dict_2045, trans_list, dis_dict, 60)

In [17]:
tot_2020_all.head()

,index,dist_origin,Pittsburgh Chateau Area,Pittsburgh Lower North Side,Pittsburgh Upper North Side,Allegheny County Northwest,Allegheny County North (remainder),Hazelwood,Squirrel Hill,Oakland,...,Greene County,Indiana County,Lawrence County,Washington County,Westmoreland County,Midtown/Uptown,Highland Park,Homewood/Lincoln-Lemington,Pittsburgh Southwest (remainder),Allegheny County Southcentral
0,1,Pittsburgh Chateau Area,3313,2181.0,1118.0,1493.0,2458.0,54.0,160.0,1238.0,...,0.0,0.0,0.0,36.0,18.0,440.0,24.0,106.0,1909.0,1687.0
1,2,Pittsburgh Lower North Side,1782,3400.0,993.0,556.0,2774.0,65.0,179.0,1415.0,...,0.0,0.0,0.0,45.0,27.0,604.0,46.0,188.0,1635.0,1633.0
2,3,Pittsburgh Upper North Side,3932,4673.0,3921.0,2976.0,10535.0,107.0,260.0,2867.0,...,0.0,0.0,0.0,28.0,17.0,1205.0,64.0,285.0,2825.0,2847.0
3,4,Allegheny County Northwest,3040,1919.0,1548.0,47603.0,14946.0,71.0,163.0,1794.0,...,0.0,0.0,0.0,125.0,4.0,582.0,20.0,111.0,2467.0,2055.0
4,5,Allegheny County North (remainder),6512,10012.0,6805.0,15690.0,596529.0,400.0,1868.0,16492.0,...,0.0,5.0,9.0,71.0,10145.0,3586.0,1361.0,9547.0,6397.0,5760.0


In [194]:
def divider_a_b(a,b):
    a2 = a.iloc[:,2:].values
    b = b.iloc[:,2:].values
    c = np.divide(a2, b, out=np.zeros_like(a2), where=b!=0)
    c = pd.DataFrame(c)
    c.columns = [str(k)+' '+v for k, v in dis_dict.items()]
    c.insert(0, 'Dist_No',  a.iloc[:,[0]])
    c.insert(1, 'District_Name',  a.iloc[:,[1]])
    return(c)

In [195]:
trans_rider_multiplier = divider_a_b(trans_2045_all, trans_2020_all)
mode_share_2020 = divider_a_b(trans_2020_all, tot_2020_all)
mode_share_2045 = divider_a_b(trans_2045_all, tot_2045_all)
mode_share_2020.to_csv('../Outputs/spc2020_inbound_pct_transit_trips.csv')
mode_share_2045.to_csv('../Outputs/spc2045_inbound_pct_transit_trips.csv')
mode_share_2020.transpose().to_csv('../Outputs/spc2020_outbound_pct_transit_trips.csv')
mode_share_2045.transpose().to_csv('../Outputs/spc2045_outound_pct_transit_trips.csv')
trans_rider_multiplier.to_csv('../Outputs/spc_future_multiplier.csv')

In [196]:
trans_rider_multiplier.head()

,Dist_No,District_Name,1 Pittsburgh Chateau Area,2 Pittsburgh Lower North Side,3 Pittsburgh Upper North Side,4 Allegheny County Northwest,5 Allegheny County North (remainder),6 Hazelwood,7 Squirrel Hill,8 Oakland,...,35 Greene County,36 Indiana County,37 Lawrence County,38 Washington County,39 Westmoreland County,40 Midtown/Uptown,41 Highland Park,42 Homewood/Lincoln-Lemington,43 Pittsburgh Southwest (remainder),44 Allegheny County Southcentral
0,1,Pittsburgh Chateau Area,1.15385,1.07216,1.10714,1.04839,0.957746,3,1.16667,1.39583,...,0,0,0,0,0,1.56522,2,1,1.14815,1.06522
1,2,Pittsburgh Lower North Side,1,1.0119,1.08824,1.07143,0.975,4,1.4,1.45679,...,0,0,0,1,0,1.64935,1.25,1.16667,1.06818,0.986486
2,3,Pittsburgh Upper North Side,1.05691,1.03256,1.14062,1.02151,0.992248,2.8,1.2,1.39464,...,0,0,0,0,0,1.48913,2,1,1.13158,1.01429
3,4,Allegheny County Northwest,1.06748,1.0367,1.09524,1.04373,0.875622,2.5,1.66667,1.79661,...,0,0,0,1,0,1.64706,2,1,1.17857,1
4,5,Allegheny County North (remainder),1.06452,1.03509,1.125,0.970588,1.028,3.83333,1.375,1.27941,...,0,0,0,0,1.04301,1.57143,0.631579,0.8375,1.16071,1


In [197]:
tot_2020_all.to_csv('../Outputs/spc2020_inbound_tot_trips.csv')
tot_2045_all.to_csv('../Outputs/spc2045_inbound_tot_trips.csv')
tot_2020_all.transpose().to_csv('../Outputs/spc2020_outbound_tot_trips.csv')
trans_2045_all.transpose().to_csv('../Outputs/spc2045_outbound_tot_trips.csv')

trans_2020_all.to_csv('../Outputs/spc2020_inbound_transit_trips.csv')
trans_2045_all.to_csv('../Outputs/spc2045_inbound_transit_trips.csv')
trans_2020_all.transpose().to_csv('../Outputs/spc2020_outbound_transit_trips.csv')
trans_2045_all.transpose().to_csv('../Outputs/spc2045_outbound_transit_trips.csv')

In [198]:
trans_rider_multiplier.head()

,Dist_No,District_Name,1 Pittsburgh Chateau Area,2 Pittsburgh Lower North Side,3 Pittsburgh Upper North Side,4 Allegheny County Northwest,5 Allegheny County North (remainder),6 Hazelwood,7 Squirrel Hill,8 Oakland,...,35 Greene County,36 Indiana County,37 Lawrence County,38 Washington County,39 Westmoreland County,40 Midtown/Uptown,41 Highland Park,42 Homewood/Lincoln-Lemington,43 Pittsburgh Southwest (remainder),44 Allegheny County Southcentral
0,1,Pittsburgh Chateau Area,1.15385,1.07216,1.10714,1.04839,0.957746,3,1.16667,1.39583,...,0,0,0,0,0,1.56522,2,1,1.14815,1.06522
1,2,Pittsburgh Lower North Side,1,1.0119,1.08824,1.07143,0.975,4,1.4,1.45679,...,0,0,0,1,0,1.64935,1.25,1.16667,1.06818,0.986486
2,3,Pittsburgh Upper North Side,1.05691,1.03256,1.14062,1.02151,0.992248,2.8,1.2,1.39464,...,0,0,0,0,0,1.48913,2,1,1.13158,1.01429
3,4,Allegheny County Northwest,1.06748,1.0367,1.09524,1.04373,0.875622,2.5,1.66667,1.79661,...,0,0,0,1,0,1.64706,2,1,1.17857,1
4,5,Allegheny County North (remainder),1.06452,1.03509,1.125,0.970588,1.028,3.83333,1.375,1.27941,...,0,0,0,0,1.04301,1.57143,0.631579,0.8375,1.16071,1


In [199]:
def find_inbound_A_to_B(spc_matrix, dist_A, dist_B):
    return(spc_matrix.loc[spc_matrix.iloc[:,0]==dist_A,].iloc[:,(1+dist_B)].iloc[0])

In [200]:
find_inbound_A_to_B(trans_rider_multiplier, 5,4)

0.9705882352941176

In [24]:
interest_dist = [17, 18, 19, 20, 21, 22, 25, 11]
co_6_mon_brad = [19, 20, 21]

In [25]:
corr_mon_brad_d = trans_2020_all.iloc[[x-1 for x in co_6_mon_brad],:]
print(corr_mon_brad_d.shape)
origin_df = corr_mon_brad_d.loc[:,(corr_mon_brad_d.sum(axis=0) != 0)]
origin_df.to_csv('sum_of_dest.csv')

(3, 46)


In [26]:
def get_top_sources(df, destination_district, district_dictionary):
    return(df.iloc[:,[0,1,destination_district+1]].sort_values(by=district_dictionary[destination_district],ascending=False))

In [27]:
get_top_sources(trans_2020_all, 19,dis_dict)

,index,dist_origin,Monroeville Pitcairn Turtle Creek
19,20,Wilkinsburg Wilkins Braddock and vicinity,256.0
18,19,Monroeville Pitcairn Turtle Creek,138.0
38,39,Westmoreland County,137.0
20,21,Wilmerding Wall North Versailles,97.0
16,17,Verona Oakmont Penn Hills,88.0
21,22,McKeesport White Oak,49.0
41,42,Homewood/Lincoln-Lemington,47.0
24,25,Munhall Whitaker Duquesne,35.0
7,8,Oakland,31.0
17,18,Plum Borough,29.0


In [28]:
def get_top_destinations(df, origin_district, district_dictionary):
    origin_df = df.iloc[[origin_district-1],2:]
    origin_df = origin_df.T
    #origin_df = origin_df[origin_df[origin_district]!=0]
    origin_df = origin_df.sort_values(by=origin_district-1,ascending=False)
    origin_df.columns = [district_dictionary[origin_district]]
    return(origin_df)

In [29]:
get_top_destinations(trans_2020_all, 19, dis_dict)

,Monroeville Pitcairn Turtle Creek
Downtown,529
Oakland,187
Monroeville Pitcairn Turtle Creek,138
Wilkinsburg Wilkins Braddock and vicinity,73
Verona Oakmont Penn Hills,37
Wilmerding Wall North Versailles,21
Homewood/Lincoln-Lemington,18
Pittsburgh East (remainder),15
Midtown/Uptown,13
East Liberty,12


In [30]:
def find_attractions(dist_list, df):
    return(df.iloc[:,[x+1 for x in dist_list]].sum())

def find_productions(dist_list, df):
    return(df.iloc[[x-1 for x in dist_list],:].sum())

In [31]:
find_attractions(co_6_mon_brad, trans_2020_all)

Monroeville Pitcairn Turtle Creek            1078.0
Wilkinsburg Wilkins Braddock and vicinity    2127.0
Wilmerding Wall North Versailles              262.0
dtype: float64

In [32]:
def get_prods_attr_trips(df, district_num, district_dictionary):
    origin_df = df.iloc[[district_num],2:].T
    origin_df.reset_index(inplace=True)

    prod_attr = df.iloc[:,[0,1,district_num+1]]
    prod_attr = prod_attr.merge(origin_df, left_on='dist_origin', right_on='index', how='left')
    prod_attr = prod_attr[['dist_origin', district_dictionary[district_num], district_num]]
    prod_attr.columns = ['District', 'Productions', 'Attractions']
    prod_attr['Total_Trips'] = (prod_attr.Productions + prod_attr.Attractions)/2.0
    prod_attr
    return(prod_attr)

In [33]:
print(dis_dict[19])
print(dis_dict[20])
print(dis_dict[21])

Monroeville Pitcairn Turtle Creek
Wilkinsburg Wilkins Braddock and vicinity
Wilmerding Wall North Versailles


In [34]:
get_prods_attr_trips(trans_2020_all, 19, dis_dict)

,District,Productions,Attractions,Total_Trips
0,Pittsburgh Chateau Area,1.0,33,17
1,Pittsburgh Lower North Side,2.0,51,26.5
2,Pittsburgh Upper North Side,1.0,9,5
3,Allegheny County Northwest,1.0,12,6.5
4,Allegheny County North (remainder),17.0,94,55.5
5,Hazelwood,6.0,31,18.5
6,Squirrel Hill,24.0,232,128
7,Oakland,31.0,2578,1304.5
8,South Side,7.0,78,42.5
9,Shadyside,13.0,284,148.5


In [35]:
get_prods_attr_trips(trans_2045_all, 19, dis_dict)

,District,Productions,Attractions,Total_Trips
0,Pittsburgh Chateau Area,0.0,32,16
1,Pittsburgh Lower North Side,1.0,47,24
2,Pittsburgh Upper North Side,1.0,8,4.5
3,Allegheny County Northwest,0.0,10,5
4,Allegheny County North (remainder),13.0,80,46.5
5,Hazelwood,8.0,86,47
6,Squirrel Hill,29.0,349,189
7,Oakland,46.0,2930,1488
8,South Side,8.0,89,48.5
9,Shadyside,13.0,262,137.5


In [36]:
def get_trip_matrix(df, dist_list, district_dictionary):
    first_dist = dist_list[0]
    matrix = get_prods_attr_trips(df, first_dist, district_dictionary)
    matrix = matrix.rename({'Total_Trips': district_dictionary[first_dist]}, axis='columns')
    for d in dist_list[1:]:
        matrix[dis_dict[d]] = get_prods_attr_trips(df, d, district_dictionary).Total_Trips
    matrix = matrix.drop(columns=['Productions', 'Attractions'])
    return(matrix)

In [37]:
matrix_2020 = get_trip_matrix(trans_2020_all, co_6_mon_brad, dis_dict)
matrix_2020.head(2)

,District,Monroeville Pitcairn Turtle Creek,Wilkinsburg Wilkins Braddock and vicinity,Wilmerding Wall North Versailles
0,Pittsburgh Chateau Area,17,3.5,1
1,Pittsburgh Lower North Side,26.5,5.5,1


In [38]:
matrix_2045 = get_trip_matrix(trans_2045_all, co_6_mon_brad, dis_dict)
matrix_2045.head(15)

,District,Monroeville Pitcairn Turtle Creek,Wilkinsburg Wilkins Braddock and vicinity,Wilmerding Wall North Versailles
0,Pittsburgh Chateau Area,16,2.5,0.5
1,Pittsburgh Lower North Side,24,6,0.5
2,Pittsburgh Upper North Side,4.5,4,0
3,Allegheny County Northwest,5,2,0
4,Allegheny County North (remainder),46.5,9.5,1
5,Hazelwood,47,15.5,17
6,Squirrel Hill,189,97.5,12
7,Oakland,1488,227.5,66.5
8,South Side,48.5,13,11
9,Shadyside,137.5,38.5,5


In [39]:
matrix_2020.to_csv('spc_trips_2020_all.csv')
matrix_2045.to_csv('spc_trips_2045_all.csv')

## Route config

We want to find out which routes start and end in districts

In [40]:
route = pd.read_csv('../Data/gtfs/routes.txt', sep=',')
route = route[["route_id", "route_short_name", "route_long_name"]]
route.head()

,route_id,route_short_name,route_long_name
0,001-183,1,Freeport Road
1,002-183,2,Mount Royal
2,004-183,4,Troy Hill
3,006-183,6,Spring Hill
4,007-183,7,Spring Garden


In [41]:
spc_route_join = pd.read_csv('..\Outputs\spc_routes_joined_consolid.csv')
spc_route_join.head()

,District,District Name,StopID,StopName,Routes Impacted
0,7.0,Squirrel Hill,E54470,TILBURY AVE OPP ALDERSON ST,"65, 74"
1,20.0,Wilkinsburg Wilkins Braddock and vicinity,E54550,TRENTON AVE AT FRANKLIN AVE,71C
2,20.0,Wilkinsburg Wilkins Braddock and vicinity,E54580,TRENTON AVE AT PENN AVE,71C
3,20.0,Wilkinsburg Wilkins Braddock and vicinity,E54585,TRENTON AVE AT PENN AVE FS,71C
4,20.0,Wilkinsburg Wilkins Braddock and vicinity,E54590,TRENTON AVE OPP REBECCA AVE NS,71C


In [42]:
def routefinder(routename, searchstring):
    '''The list of routes come in the form 69, 67 etc
       However we need to also account for P routes that may overlap
        such as 67 and P67 and keep them distinct'''

    tpatt = re.compile('^{0}| {0}'.format(routename))
    if tpatt.search(searchstring):
        print('Found {}'.format(routename))
        return(True)
    else:
        return(False)

In [43]:
routefinder('67', 'P69, 67')

Found 67


True

In [44]:
def find_routes_dist(dist_no, spc_route_matcher):
    routeset = set()
    [routeset.update(r) for r in spc_route_join[spc_route_matcher['District']==dist_no]['Routes Impacted'].str.split()]
    routeset = {d.replace(',','') for d in routeset}
    return(routeset)

In [45]:
district_route_dict = {dist_no : find_routes_dist(dist_no, spc_route_join) for dist_no in range(1,45)}

In [46]:
def route_matcher(routes, districtnum, district_dict_ref):
    return([int(r in district_dict_ref[districtnum]) for r in routes])

First we wil add the trip counts before the district indicator

In [47]:
url = 'https://data.wprdc.org/dataset/d1eb0fcd-ba60-4407-9969-ceef464d0c00/resource/d42a9392-f95c-45f7-840f-3829ed21e03a/download/schedule_daily_agg.csv'
trip_count = pd.read_csv(url)

In [48]:
trip_count[(trip_count.PickID == 1909) & 
         #  (trip_count.DayType == 'Weekday') & 
           (trip_count.RouteCode ==  '67')]

,id,PickID,Date,DayType,Route,Service.Context,Trp.Dist.,Trip.Count,Trip_Mins,Schedule_Join,RouteCode
89402,89403,1909,2019-09-01,Sunday,067,Base,356.506,26,1326.84,Base_Sunday,67
89462,89463,1909,2019-09-02,Holiday1,067,Holiday,356.506,26,1326.84,Holiday_Holiday1,67
89537,89538,1909,2019-09-03,Weekday,067,Base,1015.077,59,4278.30,Base_Weekday,67
89636,89637,1909,2019-09-04,Weekday,067,Base,1015.077,59,4278.30,Base_Weekday,67
89735,89736,1909,2019-09-05,Weekday,067,Base,1015.077,59,4278.30,Base_Weekday,67
...,...,...,...,...,...,...,...,...,...,...,...
96368,96369,1909,2019-11-19,Weekday,067,Base,1015.077,59,4278.30,Base_Weekday,67
96467,96468,1909,2019-11-20,Weekday,067,Base,1015.077,59,4278.30,Base_Weekday,67
96566,96567,1909,2019-11-21,Weekday,067,Base,1015.077,59,4278.30,Base_Weekday,67
96665,96666,1909,2019-11-22,Weekday,067,Base,1015.077,59,4278.30,Base_Weekday,67


In [49]:
pick = pd.read_csv('https://data.wprdc.org/dataset/b401859c-412b-4cb6-ad88-a4183b83183d/resource/3f789a37-d02b-4f2e-9212-3b824fb06678/download/paac_pick_lookup.csv')
pick = pick.dropna(axis='rows')
pick.tail()

,pickID,pickStart,pickEnd
21,1811,11/18/2018,3/16/2019
22,1903,3/17/2019,6/15/2019
23,1906,6/16/2019,8/31/2019
24,1909,9/1/2019,11/23/2019
25,1911,11/24/2019,3/21/2020


In [50]:
def find_trip_count(route, trip_df, daytype, pickno):
    #print(route)
    common = trip_df[(trip_df['PickID'] == pickno) &
              (trip_df['DayType'] == daytype) & 
              (trip_df['RouteCode'] == route)]['Trip.Count']
    if common.empty:
       # print('No match found')
        return(0)
    else:
        return(common.mode().iloc[0])

In [51]:
routetrip_wkdy = [find_trip_count(r, trip_count, 'Weekday', 1909) for r in route.route_short_name]
routetrip_sat = [find_trip_count(r, trip_count, 'Saturday', 1909) for r in route.route_short_name]
routetrip_sun = [find_trip_count(r, trip_count, 'Sunday', 1909) for r in route.route_short_name]

In [52]:
route_ref = route.copy()
route_ref['wkday_trips'] = routetrip_wkdy
route_ref['sat_trips'] = routetrip_sat
route_ref['sun_trips'] = routetrip_sun

In [53]:
route_spc_ref = route_ref
for dist_no in range(1,45):
    reference = route_matcher(route.route_short_name, dist_no, district_route_dict)
    route_spc_ref[str(dist_no)] = reference

route_spc_ref.head(2)

,route_id,route_short_name,route_long_name,wkday_trips,sat_trips,sun_trips,1,2,3,4,...,35,36,37,38,39,40,41,42,43,44
0,001-183,1,Freeport Road,48,30,26,0,1,1,0,...,0,0,0,0,1,0,0,1,0,0
1,002-183,2,Mount Royal,53,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
route_spc_ref.to_csv('../Outputs/route_to_district_tb.csv')

In [55]:
trip_fall19 = pd.read_csv('../Data/gtfs/trips.txt', sep=',')
grouped_df = trip_fall19.groupby(["route_id", "service_id", "trip_headsign", "direction_id"])
grouped_df = grouped_df.agg({"trip_id":"nunique"}).reset_index()
grouped_df[grouped_df.route_id.str.contains('69')]

,route_id,service_id,trip_headsign,direction_id,trip_id
373,069-183,2003-Liberty-Saturday-77,Inbound-(IX)-TRAFFORD TO WILKINSBURG,1,11
374,069-183,2003-Liberty-Saturday-77,Outbound-(OX)-TRAFFORD,0,11
375,069-183,2003-Liberty-Sunday-15,Inbound-(IX)-TRAFFORD TO WILKINSBURG,1,10
376,069-183,2003-Liberty-Sunday-15,Outbound-(OX)-TRAFFORD,0,10
377,069-183,2003-Liberty-Weekday-35,Inbound-(IF)-TRAFFORD PITTSBURGH VIA OAKLAND,1,6
378,069-183,2003-Liberty-Weekday-35,Inbound-(IH)-TRAFFORD PITTSBURGH VIA OAKLAND,1,17
379,069-183,2003-Liberty-Weekday-35,Inbound-(IZ)-TRAFFORD OAKLAND,1,4
380,069-183,2003-Liberty-Weekday-35,Outbound-(OH)-TRAFFORD - HAYMAKER VILLAGE VIA ...,0,24
618,P69-183,2003-Liberty-Weekday-35,Inbound-(IF)-TRAFFORD FLYER PITTSBURGH VIA EAS...,1,4
619,P69-183,2003-Liberty-Weekday-35,Inbound-(IZ)-TRAFFORD FLYER,1,4


In [56]:
def find_dist_ratio(dist_A, dist_B, dist_route_ref_dict):
    overlap_routes = dist_route_ref_dict[dist_A].intersection(dist_route_ref_dict[dist_B])
    if len(overlap_routes) > 0:
        ratiodict = {r: find_trip_count(r, trip_count, 'Weekday', 1909) for r in overlap_routes}
        return(ratiodict)
    else:
        return({})

In [57]:
find_dist_ratio(17, 19, district_route_dict)

{'77': 67}

In [58]:
def find_dist_spc(route_no, dist_A, dist_B, dist_route_ref_dict):
    overlap_routes = dist_route_ref_dict[dist_A].intersection(dist_route_ref_dict[dist_B])
    if route_no in overlap_routes:
        return(route_ref.loc[route_ref['route_short_name'] == route_no, ['wkday_trips']].iloc[0]['wkday_trips'])
    else:
        return(0)

In [231]:
find_dist_spc('77', 17, 19, district_route_dict)

67

We make a new matrix of all 1935 44 by 44 district path combinations and indicate route trip counts in columns

In [59]:
dist_A = list(range(1,45))
dist_B = dist_A

spc_dist_full = pd.MultiIndex.from_product([dist_A, dist_B], names = ["dist_A", "dist_B"])
spc_dist_full = pd.DataFrame(index = spc_dist_full).reset_index()

In [60]:
dist_tuples = list(spc_dist_full.itertuples(index=False, name=None))

In [61]:
spc_from_to_routes = route.copy()
for t in dist_tuples:
    route_tuple_col = [find_dist_spc(r, t[0], t[1], district_route_dict) for r in route.route_short_name]
    spc_from_to_routes['d{0}-{1}'.format(t[0],t[1])] = route_tuple_col
spc_from_to_routes.head()

,route_id,route_short_name,route_long_name,d1-1,d1-2,d1-3,d1-4,d1-5,d1-6,d1-7,...,d44-35,d44-36,d44-37,d44-38,d44-39,d44-40,d44-41,d44-42,d44-43,d44-44
0,001-183,1,Freeport Road,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,002-183,2,Mount Royal,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,004-183,4,Troy Hill,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,006-183,6,Spring Hill,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,007-183,7,Spring Garden,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
#spc_from_to_routes.to_csv('../Outputs/dist_from_to_routes.csv')

In [63]:
btb_dist = [11,25]+list(range(17,24))

In [64]:
def make_interchange(dist_no_list, clean=True):
    spc_dist_mtx = pd.MultiIndex.from_product([dist_A, dist_no_list], names = ["dist_A", dist_no_list[0]])
    spc_dist_mtx = pd.DataFrame(index = spc_dist_mtx).reset_index()

    b_tuples = list(spc_dist_mtx
                    .itertuples(index=False, name=None))
    
    b_from_to_routes = route[['route_short_name']].copy()
    for t in b_tuples:
        route_tuple_coln = [find_dist_spc(r, t[0], t[1], district_route_dict) for r in route.route_short_name]
        b_from_to_routes['d{0}-{1}'.format(t[0],t[1])] = route_tuple_coln

    b_from_to_routes = b_from_to_routes[(b_from_to_routes.iloc[:,1:].T != 0).any()]
    b_from_to_routes = b_from_to_routes.transpose()
    btb_coln = b_from_to_routes.iloc[0] #grab the first row for the header
    b_from_to_routes = b_from_to_routes[1:] #take the data less the header row
    b_from_to_routes.columns = btb_coln
    if clean:
        b_from_to_routes = b_from_to_routes[(b_from_to_routes.iloc[:,1:].T != 0).any()]
    return(b_from_to_routes)

In [227]:
def make_interchange_counts(dist_no_list, spcMatrix, clean=True, routeNew='', routeNewTrips=0):
    spc_dist_mtx = pd.MultiIndex.from_product([dist_A, dist_no_list], names = ["dist_A", dist_no_list[0]])
    spc_dist_mtx = pd.DataFrame(index = spc_dist_mtx).reset_index()

    b_tuples = list(spc_dist_mtx
                    .itertuples(index=False, name=None))
    
    b_from_to_routes = route[['route_short_name']].copy()
    for t in b_tuples:
        route_tuple_coln = [find_dist_spc(r, t[0], t[1], district_route_dict) for r in route.route_short_name]
        b_from_to_routes[t] = route_tuple_coln
    b_from_to_routes = b_from_to_routes[(b_from_to_routes.iloc[:,1:].T != 0).any()]
    #print(b_from_to_routes)
    b_from_to_routes = b_from_to_routes.transpose()
    btb_coln = b_from_to_routes.iloc[0] #grab the first row for the header
    b_from_to_routes = b_from_to_routes[1:] #take the data less the header row
    b_from_to_routes.columns = btb_coln
    if clean:
        b_from_to_routes = b_from_to_routes[(b_from_to_routes.iloc[:,1:].T != 0).any()]
    print(b_from_to_routes.index)
    sumColn = [find_inbound_A_to_B(spcMatrix, t[0],t[1]) for t in b_from_to_routes.index]
    multSPC = [find_inbound_A_to_B(trans_rider_multiplier, t[0],t[1]) for t in b_from_to_routes.index]
    b_from_to_routes[spcMatrix.name] = sumColn
    #b_from_to_routes['spc45'] = sum2045
    b_from_to_routes['Increase 20-45'] = multSPC
    return(b_from_to_routes)

In [226]:
make_interchange_counts([19], trans_2020_all, clean=True)

Index([ (7, 19),  (8, 19), (10, 19), (12, 19), (13, 19), (15, 19), (17, 19),
       (18, 19), (19, 19), (20, 19), (21, 19), (39, 19), (40, 19), (42, 19)],
      dtype='object')


route_short_name,59,67,69,77,P12,P67,P68,P69,spc20,spc45,multSPC
"(7, 19)",0,59,49,0,0,0,0,0,24.0,29.0,1.208333
"(8, 19)",0,59,49,0,0,0,0,0,31.0,46.0,1.483871
"(10, 19)",0,0,0,67,38,19,49,12,13.0,13.0,1.000000
"(12, 19)",0,59,49,67,38,19,49,12,7.0,25.0,3.571429
"(13, 19)",0,0,0,67,38,19,49,12,18.0,19.0,1.055556
"(15, 19)",0,0,0,67,38,19,49,12,22.0,22.0,1.000000
"(17, 19)",0,0,0,67,0,0,0,0,88.0,82.0,0.931818
"(18, 19)",0,0,0,67,38,0,0,0,29.0,30.0,1.034483
"(19, 19)",38,59,49,67,38,19,49,12,138.0,127.0,0.920290
"(20, 19)",38,59,49,0,38,19,49,12,256.0,243.0,0.949219


In [66]:
#for d in btb_dist:
#    distdf = make_interchange([d])
#    distdf.to_csv('../Outputs/dist{0}_interchange.csv'.format(d))

In [67]:
route_counts = pd.read_csv('https://data.wprdc.org/datastore/dump/12bb84ed-397e-435c-8d1b-8ce543108698')
route_counts

,route,ridership_route_code,route_full_name,current_garage,mode,month_start,year_month,day_type,avg_riders,day_count
0,1,001,1 - FREEPORT ROAD,Ross,Bus,2017-01-01,201701,SAT.,970,4
1,4,004,4 - TROY HILL,Ross,Bus,2017-01-01,201701,SAT.,218,4
2,6,006,6 - SPRING HILL,Ross,Bus,2017-01-01,201701,SAT.,496,4
3,8,008,8 - PERRYSVILLE,Ross,Bus,2017-01-01,201701,SAT.,1480,4
4,11,011,11 - FINEVIEW,Ross,Bus,2017-01-01,201701,SAT.,208,4
...,...,...,...,...,...,...,...,...,...,...
8815,RED,RED,RED LINE - CASTLE SHANNON,South Hills Village,Light Rail,2020-03-01,202003,WEEKDAY,7017,22
8816,SLVR,SLVR,SILVER LINE - LIBRARY,South Hills Village,Light Rail,2020-03-01,202003,WEEKDAY,3961,22
8817,MI,000,MON INCLINE,Incline,Incline,2020-03-01,202003,SAT.,706,4
8818,MI,000,MON INCLINE,Incline,Incline,2020-03-01,202003,SUN.,592,5


In [68]:
stop_counts = pd.read_csv('https://data.wprdc.org/dataset/ece64ad3-05eb-46dd-ba38-c83b5373812f/resource/3115c0b9-b48a-49aa-8e39-fd318eb62c04/download/busstopusagebyroute.csv')

In [69]:
stop_counts.head(2)

,STOP_ID,ROUTE,STOP_ROUTE,STOP_NAME,CLEVER_ID,LATITUDE,LONGITUDE,ALL_ROUTES,SHELTER,STOP_TYPE,...,1806_ON,1806_OFF,1809_ON,1809_OFF,1811_ON,1811_OFF,1903_ON,1903_OFF,1906_ON,1906_OFF
0,P03090,P1,P03090_P1,SMITHFIELD ST AT SIXTH AVE,7280.0,40.441550,-79.997761,"39, 40, 44, 87, P1, P2, P68, P71",Envision Downtown,Bus,...,3338.2,1706.2,3534.3,1800.3,3117.1,1541.7,3434.1,1809.6,3615.6,1829.3
1,P02325,P1,P02325_P1,LIBERTY AVE AT 10TH ST,7279.0,40.443739,-79.995153,"39, 40, 44, 87, P1, P2, P68, P71",Envision Downtown,Bus,...,893.6,2300.3,994.7,2429.7,882.7,2234.2,892.4,2269.3,719.5,2050.9


In [74]:
spc_apc = pd.read_csv('../Data/joined-spc-rt.csv')

In [85]:
spc_apc['Dist_No'] = pd.to_numeric(spc_apc['Dist_No'], errors='coerce')

In [88]:
spc_apc[spc_apc['Dist_No'].isin(btb_dist)]

,Dist_No,Dist_Name,Route,APC_ON_SUM,APC_OFF_SUM,STOP_CNT
0,20.0,Wilkinsburg Wilkins Braddock and vicinity,71C,344,337,19
1,18.0,Plum Borough,P16,24,21,29
2,18.0,Plum Borough,77,27,24,35
3,17.0,Verona Oakmont Penn Hills,P16,287,285,98
6,17.0,Verona Oakmont Penn Hills,P17,87,5,31
...,...,...,...,...,...,...
349,22.0,McKeesport White Oak,P7,144,124,12
365,25.0,Baldwin Brentwood Whitehall West Mifflin,56,13,7,2
366,25.0,Baldwin Brentwood Whitehall West Mifflin,53L,5,1,1
496,11.0,Waterfront (West Homestead Homestead Munhall(p...,61C,527,559,19


In [104]:
route_spc_ref.head()

,route_id,route_short_name,route_long_name,wkday_trips,sat_trips,sun_trips,1,2,3,4,...,35,36,37,38,39,40,41,42,43,44
0,001-183,1,Freeport Road,48,30,26,0,1,1,0,...,0,0,0,0,1,0,0,1,0,0
1,002-183,2,Mount Royal,53,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,004-183,4,Troy Hill,46,26,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,006-183,6,Spring Hill,63,32,32,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,007-183,7,Spring Garden,12,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
trans_2020_all.head()

,index,dist_origin,Pittsburgh Chateau Area,Pittsburgh Lower North Side,Pittsburgh Upper North Side,Allegheny County Northwest,Allegheny County North (remainder),Hazelwood,Squirrel Hill,Oakland,...,Greene County,Indiana County,Lawrence County,Washington County,Westmoreland County,Midtown/Uptown,Highland Park,Homewood/Lincoln-Lemington,Pittsburgh Southwest (remainder),Allegheny County Southcentral
0,1,Pittsburgh Chateau Area,65,97.0,28.0,62.0,71.0,3.0,6.0,144.0,...,0.0,0.0,0.0,0.0,0.0,46.0,1.0,5.0,54.0,46.0
1,2,Pittsburgh Lower North Side,89,84.0,34.0,14.0,120.0,3.0,10.0,243.0,...,0.0,0.0,0.0,1.0,1.0,77.0,4.0,12.0,88.0,74.0
2,3,Pittsburgh Upper North Side,123,215.0,64.0,93.0,258.0,5.0,10.0,261.0,...,0.0,0.0,0.0,0.0,0.0,92.0,2.0,11.0,76.0,70.0
3,4,Allegheny County Northwest,163,109.0,42.0,343.0,201.0,4.0,6.0,177.0,...,0.0,0.0,0.0,1.0,0.0,68.0,1.0,5.0,56.0,41.0
4,5,Allegheny County North (remainder),62,171.0,40.0,68.0,1357.0,6.0,24.0,816.0,...,0.0,0.0,0.0,0.0,93.0,154.0,38.0,160.0,56.0,38.0


In [102]:
p68 = spc_from_to_routes[spc_from_to_routes['route_short_name']=='P68'].transpose()
p68[p68.iloc[:,0]!=0]

,86
route_id,P68-183
route_short_name,P68
route_long_name,Braddock Hills Flyer
d10-10,49
d10-12,49
...,...
d42-15,49
d42-19,49
d42-20,49
d42-21,49


In [107]:
def get_top_sources(df, destination_district, district_dictionary):
    return(df.iloc[:,[0,1,destination_district+1]].sort_values(by=district_dictionary[destination_district],ascending=False))
def get_top_destinations(df, origin_district, district_dictionary):
    origin_df = df.iloc[[origin_district-1],2:]
    origin_df = origin_df.T
    #origin_df = origin_df[origin_df[origin_district]!=0]
    origin_df = origin_df.sort_values(by=origin_district-1,ascending=False)
    origin_df.columns = [district_dictionary[origin_district]]
    return(origin_df)